Setup env

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/LicheeRV_Nano_Resources/tpu-mlir_v1.9.beta.0-110-gfdb73ce31-20240719.tar.gz tpu-mlir.tar.gz
!mkdir tpu_mlir
!tar -xzf tpu-mlir.tar.gz -C tpu_mlir --strip-components=1
!mv ./tpu_mlir/$(ls ./tpu_mlir) ./tpu_mlir/source
!wget https://github.com/sophgo/tpu-mlir/releases/download/v1.14/tpu_mlir-1.14-py3-none-any.whl
!sudo add-apt-repository ppa:deadsnakes/ppa -y
!sudo apt update
!sudo apt install python3.10
!wget https://bootstrap.pypa.io/get-pip.py
!python3.10 get-pip.py
!python3.10 -m pip install tpu_mlir-1.14-py3-none-any.whl
!python3.10 -m pip install flatbuffers
!python3.10 -m pip install ultralytics
!python3.10 -m pip install onnxsim
!mkdir work
%cd work

In [ ]:
!python3.10 -m pip install onnxscript onnxruntime
!python3 -m pip install onnxscript onnxruntime

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# DQN arch
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)


In [ ]:
test_array = [-0.03209978, -0.0461358 , -0.01805152,  0.03391415]

policy_net = DQN(4, 2) # 4 - input size observation); 2 - output size (model actions)
policy_net.load_state_dict(torch.load("/content/trained.pth", map_location=torch.device('cpu')))
policy_net.eval()

# Test model on some data
print(policy_net(torch.tensor(test_array))) # Test

onnx_program = torch.onnx.dynamo_export(policy_net, torch.randn(1, 4))
onnx_program.save("/content/dqn.onnx")

In [9]:
import os

os.environ['MODEL_NAME'] = "dqn"

In [30]:
import numpy as np

# Create test input, that can be used to verify model's output on each step
with open("/content/test_input.npy", "wb") as fd:
  np.save(fd, np.array([test_array], dtype=np.float64))

np.savez("/content/test_input.npz", **{"arr_2d": np.array([test_array], dtype=np.float32)})

In [ ]:
%cd /content
!rm -rf work
!mkdir work
%cd work

In [ ]:
!export PYTHONPATH=/usr/bin/python3.10
!source ../tpu_mlir/source/envsetup.sh
!echo $MODEL_NAME
!model_transform.py --model_name $MODEL_NAME --model_def ../$MODEL_NAME.onnx --input_shapes "[[1, 4]]" --mean 0.0,0.0,0.0,0.0 --scale 1,1,1,1 --mlir $MODEL_NAME.mlir --test_input /content/test_input.npy --test_result /content/mlir_res.npz

In [48]:
out = np.load("/content/mlir_res.npz", allow_pickle=True)
print(out["layer3_1_Gemm"])

[[96.12512 95.90169]]


In [ ]:
!export PYTHONPATH=/usr/bin/python3.10
!source ../tpu_mlir/source/envsetup.sh

!model_deploy \
--mlir $MODEL_NAME.mlir \
--quant_input \
--test_input /content/test_input.npz \
--quantize bf16 \
--processor cv181x \
--tolerance 0.99,0.99 \
--model dqn.cvimodel

In [ ]:
!model_runner.py --input /content/test_input.npz --model dqn.cvimodel --output dqn_cv181x_bf16_model_outputs.npz

In [60]:
np.load("dqn_cv181x_bf16_model_outputs.npz")["layer3_1_Gemm_f32"]

array([[[[96.]],

        [[96.]]]], dtype=float32)